In [9]:
import tensorflow as tf
import joblib
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import torch
from torch.autograd import Variable

In [7]:
file = (
    '/home/vitchyr/git/rllab-rail/railrl/data/local/6-16-launch-benchmark-1d-ddpg-correct/6-16-launch-benchmark-1d-ddpg-correct_2017_06_16_19_05_45_0001--s-0'
    '/params.pkl'
)

tf.reset_default_graph()
sess = tf.InteractiveSession()

data = joblib.load(file)
policy = data['policy']
qf = data['qf']
env = data['env']
print("Policy loaded")

Policy loaded


In [8]:
def make_heat_map(eval_func, *, resolution=50, min_val=-1, max_val=1):
    linspace = np.linspace(min_val, max_val, num=resolution)
    map = np.zeros((resolution, resolution))

    for i in range(resolution):
        for j in range(resolution):
            map[i, j] = eval_func(np.array([linspace[i], linspace[j]]))
    return map

def plot_maps(old_combined=None, *heatmaps):
    import matplotlib.pyplot as plt
    combined = np.c_[heatmaps]
    if old_combined is not None:
        combined = np.r_[old_combined, combined]
    plt.figure()
    plt.imshow(combined, cmap='afmhot', interpolation='none')
    plt.show()
    return combined

In [40]:
def create_eval_fnct(qf, on_target):
    def evaluate(state_and_action):
        state, action = state_and_action[:1], state_and_action[1:]
        state = np.hstack([state, on_target])
        state = Variable(torch.from_numpy(state)).float().unsqueeze(0)
        action = Variable(torch.from_numpy(action)).float().unsqueeze(0)
        print(state.size())
        print(action.size())
        return qf(state, action)
    return evaluate

In [41]:
qf_eval = create_eval_fnct(qf, 0)


In [42]:
qf_eval(np.array([1, 1]))

torch.Size([1, 2])
torch.Size([1, 1])


RuntimeError: size mismatch, m1: [1 x 2], m2: [3 x 400] at /py/conda-bld/pytorch_1493677666423/work/torch/lib/TH/generic/THTensorMath.c:1237